In [1]:
!pip install numpy
!pip install Flask
!pip install pandas
!pip install torch
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install -U sentence-transformers -q
!pip install faiss-cpu -q
!pip install transformers
!pip install tensor

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:

import numpy as np 
import pandas as pd 
import os, warnings
warnings.simplefilter('ignore')
import time


In [4]:

from sentence_transformers import SentenceTransformer
import faiss

# Torch library
import torch

from transformers import pipeline


W1018 15:04:27.064000 25648 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [ ]:
file = 'C:/Users/user/Desktop/projects/cricket llm/final project/backend/batting_stats.csv'

df = pd.read_csv(file)

df_cleaned = df.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',])

df_cleaned = df_cleaned.dropna(subset=['Player', 'Span', 'Matches', 'Innings', 'Runs'])

df_cleaned_copy = df_cleaned.copy()

# Save the cleaned DataFrame to a CSV file
output_file = 'C:/Users/user/Desktop/projects/cricket llm/final project/backend/batting_stats_cleaned.csv'
df_cleaned_copy.to_csv(output_file, index=False)


df_cleaned_copy.head()


,S.no,Player,Format,Span,Matches,Innings,Not outs,Runs,High Score,Average,Balls faced,Strike rate,Centuries,Half-Centuries,0 runs,Unnamed: 15,4s,6s
0,1,Sachin Tendulkar,ODI,1989-2012,463,452,41,18426,200*,44.83,21367.0,86.23,49,96,20,NaN,NaN,NaN
1,2,Kumar Sangakkara (Asia/ICC/SL),ODI,2000-2015,404,380,41,14234,169,41.98,18048.0,78.86,25,93,15,NaN,NaN,NaN
2,3,Ricky Ponting (AUS/ICC),ODI,1995-2012,375,365,39,13704,164,42.03,17046.0,80.39,30,82,20,NaN,NaN,NaN
3,4,Sanath Jayasuriya (Asia/SL),ODI,1989-2011,445,433,18,13430,189,32.36,14725.0,91.2,28,68,34,NaN,NaN,NaN
4,5,Mahela Jayawardene (Asia/SL),ODI,1998-2015,448,418,39,12650,144,33.37,16020.0,78.96,19,77,28,NaN,NaN,NaN


In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
text_columns = ['S.no','Player','Format','Span','Matches','Innings','Not outs','Runs','High Score','Average','Balls faced','Strike rate','Centuries','Half-Centuries','0 runs','4s','6s']


In [9]:

all_embeddings = []

# Loop through each row and encode the text columns
for _, row in df_cleaned.iterrows():
    # Concatenate the text columns into a single string
    combined_text = ' '.join([str(row[col]) for col in text_columns])
    # Encode the combined text
    embedding = model.encode(combined_text)
    all_embeddings.append(embedding)

# Convert the list of embeddings into a numpy array
all_embeddings = np.array(all_embeddings)


In [ ]:
# Check the shape of the embeddings array
print(all_embeddings.shape)

(7325, 384)


In [ ]:
np.save('embeddings.npy', all_embeddings)

In [ ]:
import faiss
import numpy as np

embeddings = np.load('embeddings.npy')

In [13]:
# Create a FAISS index
dim = embeddings.shape[1]  # Dimensionality of your embeddings
index = faiss.IndexFlatL2(dim)  # Using L2 distance
# Add embeddings to the index
index.add(embeddings)


In [14]:
faiss.write_index(index, 'faiss_index_batting_stats')
index = faiss.read_index('faiss_index_batting_stats')


In [16]:

# Encode your query
query = "How many matches did Virat kohli played in t20?"
query_embedding = model.encode([query])[0].reshape(1, -1) 

In [17]:
# Perform the search
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Retrieve the results
print("Nearest neighbors indices:", indices)
print("Distances:", distances)

# Retrieve the most similar row
closest_row = df.iloc[indices[0][0]]
print(closest_row)


Nearest neighbors indices: [[2490 4389    6 2523 2541]]
Distances: [[0.51515347 0.68852377 0.7077631  0.7969259  0.8663709 ]]
S.no                             2491
Player            Virat Kohli (INDIA)
Format                            t20
Span                        2010-2019
Matches                            75
Innings                            70
Not outs                           20
Runs                             2633
High Score                        94*
Average                         52.66
Balls faced                    1907.0
Strike rate                    138.07
Centuries                           0
Half-Centuries                     24
0 runs                              2
Unnamed: 15                       NaN
Unnamed: 16                       NaN
Unnamed: 17                       NaN
Unnamed: 18                       NaN
4s                              247.0
6s                               71.0
Name: 2490, dtype: object


In [18]:
closest_row_df = closest_row.to_frame().T
print(closest_row_df)


      S.no               Player Format       Span Matches Innings Not outs  \
2490  2491  Virat Kohli (INDIA)    t20  2010-2019      75      70       20   

      Runs High Score Average  ... Strike rate Centuries Half-Centuries  \
2490  2633        94*   52.66  ...      138.07         0             24   

     0 runs Unnamed: 15 Unnamed: 16 Unnamed: 17 Unnamed: 18     4s    6s  
2490      2         NaN         NaN         NaN         NaN  247.0  71.0  

[1 rows x 21 columns]


In [19]:
from transformers import pipeline
import logging
from transformers import logging as hf_logging

hf_logging.set_verbosity_error()

closest_row_dict = closest_row_df.to_dict(orient='records')[0]  

query = "How many matches did Virat Kohli play in T20?"

qa_input = {
    'question': query,
    'context': str(closest_row_dict) 
}
qa_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')
response = qa_pipeline(qa_input, max_new_tokens=50, truncation=True)
answer = response['answer']
print(answer)

75


In [20]:
model.save('sentence_transformer_model')